## Libraries

In [ ]:
import os
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from natsort import natsorted
import pandas as pd
import numpy as np
import math
import warnings

warnings.filterwarnings("ignore")

## Read files

In [ ]:
def read_files(file):
    if 'txt' in file:
        with open(f'Articles/'+file, 'r') as f:
            return f.read()

In [ ]:
documents = []
for file in os.listdir('Articles'):
    documents.append(read_files(file))

In [ ]:
documents

# First Phase $:-$

## Apply tokenization

In [ ]:
token_docs = []
for document in documents:
    token_docs.append(word_tokenize(document))

## Stemming

In [ ]:
ps = PorterStemmer()

In [ ]:
stemmed_docs =[[ps.stem(term) for term in token] for token in token_docs]

In [ ]:
documents[0][:10]

In [ ]:
documents

# Second phase $:-$

### Implement function to do all steps in first phase

In [ ]:
def preprocessing(doc):
    token_docs = word_tokenize(doc)
    ps = PorterStemmer()
    prepared_doc = [ps.stem(term) for term in token_docs]
    return prepared_doc


In [ ]:
# Initialize the file no.
fileno = 1

# Initialize the dictionary.
pos_index = {}

In [ ]:
# Open files.
file_names = natsorted(os.listdir("Articles"))
print(file_names)
# For every file.
for file_name in file_names:

    # Read file contents.
    with open(f'Articles/{file_name}', 'r') as f:
        doc = f.read()
    # preprocess doc
    final_token_list = preprocessing(doc)

    # For position and term in the tokens.
    for pos, term in enumerate(final_token_list):
        # print(pos, '-->' ,term)
        
        # If term already exists in the positional index dictionary.
        if term in pos_index:
                
            # Increment total freq by 1.
            pos_index[term][0] = pos_index[term][0] + 1
            
            # Check if the term has existed in that DocID before.
            if fileno in pos_index[term][1]:
                pos_index[term][1][fileno].append(pos)
                    
            else:
                pos_index[term][1][fileno] = [pos]

        # If term does not exist in the positional index dictionary
        else:
            
            # Initialize the list.
            pos_index[term] = []
            # The total frequency is 1.
            pos_index[term].append(1)
            # The postings list is initially empty.
            pos_index[term].append({})     
            # Add doc ID to postings list.
            pos_index[term][1][fileno] = [pos]

    # Increment the file no. counter for document ID mapping             
    fileno += 1

### displays each term 

In [ ]:
pos_index

### Allow users to write phrase query 

In [ ]:
#Phrase Query
def put_query(q, display=1):
    lis = [[] for i in range(10)]
    q = preprocessing(q)
    for term in q:

        if term in pos_index.keys():
            for key in pos_index[term][1].keys():
            
                if lis[key-1] != []:
                    
                    if lis[key-1][-1] == pos_index[term][1][key][0]-1:
                        lis[key-1].append(pos_index[term][1][key][0])
                else:
                    lis[key-1].append(pos_index[term][1][key][0])
    positions = []
    if display==1:
        for pos, list in enumerate(lis, start=1):
            if len(list) == len(q):
                positions.append('document '+str(pos))
        return positions
    else:
        for pos, list in enumerate(lis, start=1):
            if len(list) == len(q):
                positions.append('doc'+str(pos))
        return positions

In [ ]:
q = 'antony brutus caeser cleopatra mercy worser'
put_query(q)

# Third phase $:-$

In [ ]:
documents = []
files = os.listdir('Articles')
for file in range(1, 11):
    documents.append(" ".join(preprocessing(read_files(str(file)+'.txt'))))

In [ ]:
all_terms = []
for doc in documents:
    for term in doc.split():
        all_terms.append(term)
all_terms = set(all_terms)

## Term Frequency
$$ tf = \frac{number of times the term appears in a document} {total number of words in the document}$$

In [ ]:
def get_tf(document):
    wordDict = dict.fromkeys(all_terms, 0)
    for word in document.split():
        wordDict[word] += 1
    return wordDict

In [ ]:
tf = pd.DataFrame(get_tf(documents[0]).values(), index=get_tf(documents[0]).keys())
for i in range(1, len(documents)):
    tf[i] = get_tf(documents[i]).values()
tf.columns = ['doc'+str(i) for i in range(1, 11)]

In [ ]:
tf

## Weighted tf(1+ log tf)

In [ ]:
def weighted_tf(x):
    if x > 0:
        return math.log10(x) + 1
    return 0

In [ ]:
w_tf = tf.copy()
for i in range(0, len(documents)):
    w_tf['doc'+str(i+1)] = tf['doc'+str(i+1)].apply(weighted_tf)

In [ ]:
w_tf

## Inverse Document Frequency
$$ idf = \frac{number of the documents in the corups} {number of documents in the corups contain the term}$$

In [ ]:
tdf = pd.DataFrame(columns=['df', 'idf'])
for i in range(len(tf)):
    in_term = w_tf.iloc[i].values.sum()

    tdf.loc[i, 'df'] = in_term

    tdf.loc[i, 'idf'] = math.log10(10 / (float(in_term)))

tdf.index=w_tf.index

In [ ]:
tdf

## TF.IDF

In [ ]:
tf_idf = w_tf.multiply(tdf['idf'], axis=0)

In [ ]:
tf_idf

## Document length

In [ ]:
def get_doc_len(col):
    return np.sqrt(tf_idf[col].apply(lambda x: x**2).sum())

In [ ]:
doc_len = pd.DataFrame()
for col in tf_idf.columns:
    doc_len.loc[0, col+'_length']= get_doc_len(col)


In [ ]:
doc_len

In [ ]:
doc_len['doc1_length'].values[0]

## Normalized TF.IDF

In [ ]:
def get_norm_tf_idf(col, x):
    try:
        return x / doc_len[col+'_length'].values[0]
    except:
        return 0

In [ ]:
norm_tf_idf = pd.DataFrame()
for col in tf_idf.columns:
    norm_tf_idf[col] = tf_idf[col].apply(lambda x : get_norm_tf_idf(col, x))

In [ ]:
norm_tf_idf

## Input Query

In [ ]:
def get_w_tf(x):
    try:
        return math.log10(x)+1
    except:
        return 0

In [ ]:
def insert_query(q):
    docs_found = put_query(q, 2)
    if docs_found == []:
        return "Not Fount"
    new_q = preprocessing(q)
    query = pd.DataFrame(index=norm_tf_idf.index)
    query['tf'] = [1 if x in new_q else 0 for x in list(norm_tf_idf.index)]
    query['w_tf'] = query['tf'].apply(lambda x : get_w_tf(x))
    product = norm_tf_idf.multiply(query['w_tf'], axis=0)
    query['idf'] = tdf['idf'] * query['w_tf']
    query['tf_idf'] = query['w_tf'] * query['idf']
    query['normalized'] = 0
    for i in range(len(query)):
        query['normalized'].iloc[i] = float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values**2))
    print('Query Details')
    print(query.loc[new_q])
    product2 = product.multiply(query['normalized'], axis=0)
    scores = {}
    for col in put_query(q, 2):
            scores[col] = product2[col].sum()
    product_result = product2[list(scores.keys())].loc[new_q]
    print()
    print('Product (query*matched doc)')
    print(product_result)
    print()
    print('product sum')
    print(product_result.sum())
    print()
    print('Query Length')
    q_len = math.sqrt(sum([x**2 for x in query['idf'].loc[new_q]]))
    print(q_len)
    print()
    print('Cosine Simliarity')
    print(product_result.sum())
    print()
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    print('Returned docs')
    for typle in sorted_scores:
        print(typle[0], end=" ")

In [ ]:
insert_query('antony brutus')